In [1]:
import time, json, requests

BASE_URL = "https://breast-cancer-api-37mk.onrender.com"
HEALTH_URL = f"{BASE_URL}/health"
PRED_URL   = f"{BASE_URL}/predict?confidence=0.5"

# Definimos un tiempo de espera en función del plan gratuito de render
def warm_up(max_wait_s=420, initial_read_to=40, max_read_to=180, sleep_s=2):
    start = time.time()
    attempt = 0
    read_to = initial_read_to
    connect_to = 6

    while True:
        attempt += 1
        try:
            r = requests.get(HEALTH_URL, timeout=(connect_to, read_to))
            ok = (r.status_code == 200 and r.json().get("status") == "ok")
            print(f"[warm-up {attempt}] HTTP {r.status_code} | elapsed={r.elapsed.total_seconds():.1f}s | ok={ok}")
            if ok:
                print("Servicio listo ✅")
                return
        except requests.exceptions.RequestException as e:
            print(f"[warm-up {attempt}] {type(e).__name__}: {e}")

        if time.time() - start > max_wait_s:
            raise TimeoutError("El servicio no quedó listo durante el warm-up.")
        # aumenta el read-timeout gradualmente hasta el tope
        read_to = min(read_to + 20, max_read_to)
        time.sleep(sleep_s)

def do_predict(nombre_caso, features, timeout_read=180):
    body = {"features": features}
    print("\n================= REQUEST =================")
    print(f"Caso: {nombre_caso}")
    print(f"POST {PRED_URL}")
    print(json.dumps(body, indent=2, ensure_ascii=False))

    try:
        resp = requests.post(PRED_URL, json=body, timeout=(6, timeout_read))
        print("\n================= RESPONSE =================")
        print(f"HTTP {resp.status_code}")
        try:
            print(json.dumps(resp.json(), indent=2, ensure_ascii=False))
        except Exception:
            print(resp.text)
    except requests.exceptions.ReadTimeout:
        print("\n❌ ReadTimeout: incrementa timeout_read o vuelve a ejecutar tras el warm-up.")
    except requests.exceptions.RequestException as e:
        print("\n❌ Error de red:", e)

In [2]:
# Payload 1 - Primera consulta 
features_maligno = {
    "mean radius": 17.99, "mean texture": 10.38, "mean perimeter": 122.8, "mean area": 1001.0,
    "mean smoothness": 0.1184, "mean compactness": 0.2776, "mean concavity": 0.3001,
    "mean concave points": 0.1471, "mean symmetry": 0.2419, "mean fractal dimension": 0.07871,
    "radius error": 1.095, "texture error": 0.9053, "perimeter error": 8.589, "area error": 153.4,
    "smoothness error": 0.006399, "compactness error": 0.04904, "concavity error": 0.05373,
    "concave points error": 0.01587, "symmetry error": 0.03003, "fractal dimension error": 0.006193,
    "worst radius": 25.38, "worst texture": 17.33, "worst perimeter": 184.6, "worst area": 2019.0,
    "worst smoothness": 0.1622, "worst compactness": 0.6656, "worst concavity": 0.7119,
    "worst concave points": 0.2654, "worst symmetry": 0.4601, "worst fractal dimension": 0.1189
}

#Ejecutamos las consultas con warm_up para "despertar" la instancia de su inactividad. 
warm_up() 
do_predict("Maligno (alto riesgo)", features_maligno)

[warm-up 1] HTTP 200 | elapsed=0.3s | ok=True
Servicio listo ✅

================= REQUEST =================
Caso: Maligno (alto riesgo)
POST https://breast-cancer-api-37mk.onrender.com/predict?confidence=0.5
{
  "features": {
    "mean radius": 17.99,
    "mean texture": 10.38,
    "mean perimeter": 122.8,
    "mean area": 1001.0,
    "mean smoothness": 0.1184,
    "mean compactness": 0.2776,
    "mean concavity": 0.3001,
    "mean concave points": 0.1471,
    "mean symmetry": 0.2419,
    "mean fractal dimension": 0.07871,
    "radius error": 1.095,
    "texture error": 0.9053,
    "perimeter error": 8.589,
    "area error": 153.4,
    "smoothness error": 0.006399,
    "compactness error": 0.04904,
    "concavity error": 0.05373,
    "concave points error": 0.01587,
    "symmetry error": 0.03003,
    "fractal dimension error": 0.006193,
    "worst radius": 25.38,
    "worst texture": 17.33,
    "worst perimeter": 184.6,
    "worst area": 2019.0,
    "worst smoothness": 0.1622,
    "wor

In [3]:
# Segunda consulta
features_benigno = {
    "mean radius": 12.0, "mean texture": 14.5, "mean perimeter": 78.0, "mean area": 450.0,
    "mean smoothness": 0.090, "mean compactness": 0.080, "mean concavity": 0.050,
    "mean concave points": 0.030, "mean symmetry": 0.190, "mean fractal dimension": 0.060,
    "radius error": 0.40, "texture error": 0.70, "perimeter error": 2.00, "area error": 20.0,
    "smoothness error": 0.0050, "compactness error": 0.020, "concavity error": 0.020,
    "concave points error": 0.010, "symmetry error": 0.015, "fractal dimension error": 0.0030,
    "worst radius": 14.5, "worst texture": 18.0, "worst perimeter": 95.0, "worst area": 650.0,
    "worst smoothness": 0.120, "worst compactness": 0.150, "worst concavity": 0.120,
    "worst concave points": 0.080, "worst symmetry": 0.300, "worst fractal dimension": 0.070
}

warm_up() 
do_predict("Benigno (bajo riesgo)", features_benigno)

[warm-up 1] HTTP 200 | elapsed=0.3s | ok=True
Servicio listo ✅

================= REQUEST =================
Caso: Benigno (bajo riesgo)
POST https://breast-cancer-api-37mk.onrender.com/predict?confidence=0.5
{
  "features": {
    "mean radius": 12.0,
    "mean texture": 14.5,
    "mean perimeter": 78.0,
    "mean area": 450.0,
    "mean smoothness": 0.09,
    "mean compactness": 0.08,
    "mean concavity": 0.05,
    "mean concave points": 0.03,
    "mean symmetry": 0.19,
    "mean fractal dimension": 0.06,
    "radius error": 0.4,
    "texture error": 0.7,
    "perimeter error": 2.0,
    "area error": 20.0,
    "smoothness error": 0.005,
    "compactness error": 0.02,
    "concavity error": 0.02,
    "concave points error": 0.01,
    "symmetry error": 0.015,
    "fractal dimension error": 0.003,
    "worst radius": 14.5,
    "worst texture": 18.0,
    "worst perimeter": 95.0,
    "worst area": 650.0,
    "worst smoothness": 0.12,
    "worst compactness": 0.15,
    "worst concavity": 0.

In [4]:
# Tercera consulta
features_intermedio = {
    "mean radius": 14.2, "mean texture": 16.0, "mean perimeter": 92.0, "mean area": 650.0,
    "mean smoothness": 0.095, "mean compactness": 0.120, "mean concavity": 0.090,
    "mean concave points": 0.060, "mean symmetry": 0.200, "mean fractal dimension": 0.062,
    "radius error": 0.55, "texture error": 0.80, "perimeter error": 2.80, "area error": 40.0,
    "smoothness error": 0.0055, "compactness error": 0.025, "concavity error": 0.025,
    "concave points error": 0.012, "symmetry error": 0.017, "fractal dimension error": 0.0035,
    "worst radius": 16.2, "worst texture": 22.0, "worst perimeter": 105.0, "worst area": 820.0,
    "worst smoothness": 0.135, "worst compactness": 0.220, "worst concavity": 0.170,
    "worst concave points": 0.110, "worst symmetry": 0.330, "worst fractal dimension": 0.075
}

warm_up() 
do_predict("Intermedio (borderline)", features_intermedio)

[warm-up 1] HTTP 200 | elapsed=0.2s | ok=True
Servicio listo ✅

================= REQUEST =================
Caso: Intermedio (borderline)
POST https://breast-cancer-api-37mk.onrender.com/predict?confidence=0.5
{
  "features": {
    "mean radius": 14.2,
    "mean texture": 16.0,
    "mean perimeter": 92.0,
    "mean area": 650.0,
    "mean smoothness": 0.095,
    "mean compactness": 0.12,
    "mean concavity": 0.09,
    "mean concave points": 0.06,
    "mean symmetry": 0.2,
    "mean fractal dimension": 0.062,
    "radius error": 0.55,
    "texture error": 0.8,
    "perimeter error": 2.8,
    "area error": 40.0,
    "smoothness error": 0.0055,
    "compactness error": 0.025,
    "concavity error": 0.025,
    "concave points error": 0.012,
    "symmetry error": 0.017,
    "fractal dimension error": 0.0035,
    "worst radius": 16.2,
    "worst texture": 22.0,
    "worst perimeter": 105.0,
    "worst area": 820.0,
    "worst smoothness": 0.135,
    "worst compactness": 0.22,
    "worst con